In [1]:
import glob
import os
from itertools import combinations, product

import numpy as np
np.random.seed(18012023)

import pandas as pd
pd.set_option('display.max_colwidth', 0)

from sklearn.metrics import pairwise_distances
import lxml.etree
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /Users/mikekestemont/anaconda3/envs/n36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [2]:
meta_df = pd.read_excel('../data/metadata_corrected.xlsx')
meta_df = meta_df[meta_df['exclude'] != 'x']
meta_df.sample(10)

,id,title,author,date,provenance,date_range,genre,subgenre,exclude
265,van_sente_brandane,Van sente Brandane,NaN,1402.5,cdrom-mnl,1380-1425,Epiek,Heiligenleven,NaN
63,florigout_fragm_l,Florigout,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Ridder,NaN
111,loyhier_en_malaert_fragm_a,Loyhier en Malaert,NaN,1375.0,cdrom-mnl,1350-1400,Epiek,Karel,NaN
86,historie_van_gaver_capeel,Historie van Gaver Capeel,NaN,1510.0,cdrom-mnl,1500-1520,Epiek,Ridder,NaN
167,Reinout van Montalbaen,Reinout van Montalbaen,NaN,1288.0,CG1,1300-1276,Epiek,Karel,NaN
210,spiegel_der_jongers,Spiegel der jongers,Lambertus Goetman,1515.0,cdrom-mnl,1510-1520,Epiek,Didactiek,NaN
209,spel_van_de_antichrist,Spel van de Antichrist,NaN,1425.0,cdrom-mnl,1400-1450,Dramatiek,NaN,NaN
123,madelgijs_fragm_p,Madelgijs,NaN,1387.5,cdrom-mnl,1375-1400,Epiek,Karel,NaN
165,queeste_van_den_grale,Queeste van den Grale,NaN,1325.0,cdrom-mnl,1300-1350,Epiek,Arthur,NaN
238,strofische_gedichten,Strofische gedichten,Hadewijch,1350.0,cdrom-mnl,1340-1360,Lyriek,NaN,NaN


In [3]:
def get_verse_groups(verses, size=2):
    for i in range(len(verses) - (size)):
        yield ' '.join(verses[i : i + size] )

In [4]:
GROUP_SIZE = 2

titles, tokens, lemmas = [], [], []

for title, group in tqdm(meta_df.groupby('title')):
    work_tokens, work_lemmas = [], []
    
    for id_ in sorted(group['id']):
        try:
            tree = lxml.etree.parse(f'../data/xml/{id_}.xml')
        except OSError:
            print(f'- Could not load {id_}')
            continue
        
        for line_node in tree.iterfind('//l'):
            tokens_ = line_node.attrib['tokens'].split()
            lemmas_ = []
            
            lemma_tags = [l.text for l in line_node.iterfind('.//lemma')]
            pos_tags = [p.text for p in line_node.iterfind('.//pos')]
            
            for lemma, pos in zip(lemma_tags, pos_tags):
                for l, p in zip(lemma.split('+'), pos.split('+')):
                        if p == 'n(prop)':
                            lemmas_.append('n(prop)')
                        else:
                            lemmas_.append(l)
        
            work_tokens.append(tokens_)
            work_lemmas.append(lemmas_)
    
    verse_tokens = [' '.join(v) for v in work_tokens]
    verse_lemmas = [' '.join(v) for v in work_lemmas]

    verse_group_tokens = list(get_verse_groups(verse_tokens, size=GROUP_SIZE))
    verse_group_lemmas = list(get_verse_groups(verse_lemmas, size=GROUP_SIZE))

    tokens.extend(verse_group_tokens)
    lemmas.extend(verse_group_lemmas)
    titles.extend([title] * len(verse_group_lemmas))

100%|██████████| 205/205 [00:30<00:00,  6.74it/s]


In [5]:
df = pd.DataFrame(zip(titles, tokens, lemmas), columns=('title', 'tokens', 'lemmas'))
df[df['title'] == 'Karel ende Elegast']

,title,tokens,lemmas
376150,Karel ende Elegast,FRaeye historie ende al waer Mach ic v tellen hoort naer,n(prop) historie en al waar mogen ik gij tellen horen naar
376151,Karel ende Elegast,Mach ic v tellen hoort naer Het was op enen auontstont,mogen ik gij tellen horen naar het zijn op een avondstond
376152,Karel ende Elegast,Het was op enen auontstont Dat karel slapen begonde,het zijn op een avondstond dat karel slapen beginnen
376153,Karel ende Elegast,Dat karel slapen begonde Tengelem op den rijn,dat karel slapen beginnen n(prop) op de n(prop)
376154,Karel ende Elegast,Tengelem op den rijn Dlant was alle gader sijn,n(prop) op de n(prop) het land zijn al gader zijn
...,...,...,...
377507,Karel ende Elegast,Dies dancti onsen heere Die coninc gaf hem eggerics wijf,dat danken hij ons heer de koning geven hij n(prop) wijf
377508,Karel ende Elegast,Die coninc gaf hem eggerics wijf Si waren tsamen al haer lijf,de koning geven hij n(prop) wijf zij zijn tezamen al haar lijf
377509,Karel ende Elegast,Si waren tsamen al haer lijf Dus moet god al onse saken,zij zijn tezamen al haar lijf dus moeten god al ons zaak
377510,Karel ende Elegast,Dus moet god al onse saken Voor onse doot te goede maken,dus moeten god al ons zaak voor ons dood te goed maken


In [6]:
vec = TfidfVectorizer(max_features=5000, min_df=2, token_pattern=r"(?u)\b\w+\b")
X = vec.fit_transform(df['lemmas'])
X.shape

(920006, 5000)

In [7]:
sorted(df['title'].unique())

['AB recht ende averecht',
 'Alexanders geesten',
 'Alexiuslegende',
 'Anatomie van de mens',
 'Antwerps liedboek',
 'Arturs doet',
 'Aubri de Borgengoen',
 'Baghynken van Parys',
 'Barlaam en Josaphat',
 'Beatrijs',
 'Bediedenisse van der missen',
 'Beerte metten breden voeten',
 'Beginsel der oorlogen van Luyck',
 'Berlijnse liederenhandschrift',
 'Boec Exemplaer',
 'Boec van den houte',
 'Boec vander wraken',
 'Boeve van Hamtone',
 'Borchgrave van Couchi',
 'Borchgravinne van Vergi (1)',
 'Borchgravinne van Vergi (2)',
 'Boudewijn van Seborch',
 'Brabantsche yeesten (B1-5)',
 'Brabantsche yeesten (B6)',
 'Brabantsche yeesten (B7)',
 'Buskenblazer',
 "Chanson d'Aspremont",
 'Clausule vander bible',
 'Collectief lunarium I',
 'Collectief lunarium II',
 'Collectief lunarium III',
 'Commentaar op het Hooglied',
 'Cyromanchie van den pape van den Hamme',
 'Der kerken claghe',
 'Der leken spieghel',
 'Der mannen ende vrouwen heimelijcheit',
 'Der minnen loep',
 'Der naturen bloeme',
 'Der

In [8]:
def nearest_verses(title1, title2, base_df, vectorizer, prop_filt=2):
    A = base_df[base_df['title'] == title1]
    B = base_df[base_df['title'] == title2]
    
    AX = vec.transform(A['lemmas'])
    BX = vec.transform(B['lemmas'])
    
    nearest = []
    for i, ax in enumerate(AX):
        distances = pairwise_distances(ax, BX, metric='cosine').flatten()
        top_i = distances.argsort()[0]
        nearest.append((A.iloc[i]['tokens'], B.iloc[top_i]['tokens'],
                        A.iloc[i]['lemmas'], B.iloc[top_i]['lemmas'],
                        A.iloc[i]['title'], B.iloc[top_i]['title'],
                        distances[top_i]))

    nearest = pd.DataFrame(nearest,
                           columns=['tokens1', 'tokens2', 'lemmas1', 'lemmas2',
                                    'title1', 'title2', 'distance'])

    nearest = nearest[nearest['title1'] != nearest['title2']]
    nearest = nearest[
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) & \
                      (nearest['lemmas1'].str.count('n\(prop\)') < prop_filt) 
                     ]
    return nearest

In [9]:
nearest_df = nearest_verses('Karel ende Elegast', 'Moriaen',
                            base_df=df, vectorizer=vec, prop_filt=2)
nearest_df = nearest_df.sort_values(by='distance')
nearest_df.head(50)

,tokens1,tokens2,lemmas1,lemmas2,title1,title2,distance
729,Dies is leden die derde dach Dat ic voer om mijn beiach,Nu es leden die derde dach Dat ic voer om min bejach,die zijn lijden de derde dag dat ik varen om mijn bejag,nu zijn lijden de derde dag dat ik varen om mijn bejag,Karel ende Elegast,Moriaen,0.044827
460,In dien dat ghi mi maket vroet Wat nootsaken dat v doet,Ende segt mi ende maect mi vroet Wat noetsaken dat u doet,in die dat gij ik maken vroed wat noodzaak dat uw doen,en zeggen ik en maken ik vroed wat noodzaak dat gij doen,Karel ende Elegast,Moriaen,0.143605
1259,Ic come v heden te ghenaden Van allen minen misdaden,Want ic come u te genaden Van allen minen mesdaden,ik komen gij heden te genade van al mijn misdaad,want ik komen gij te genade van al mijn misdaad,Karel ende Elegast,Moriaen,0.168828
461,Wat nootsaken dat v doet Dat ghi hier quaemt te nacht,Wat noetsaken dat u doet Dat gi hier nu comt gereden,wat noodzaak dat uw doen dat gij hier komen te nacht,wat noodzaak dat gij doen dat gij hier nu komen rijden,Karel ende Elegast,Moriaen,0.233060
372,Dat mi een man dwinghen soude Van dinghen die ic niet en woude,Dan mi een ridder dwingen soude Van dies ic doen nine woude,dat ik een man dwingen zullen van ding die ik niet ne willen,dan ik een ridder dwingen zullen van die ik doen ne niet willen,Karel ende Elegast,Moriaen,0.242071
1195,Dat ghi segt ende brenct voort Die ondaet ende die moort,Die grote ondaet ende die mort Hi sprac te sinen lieden vort,dat gij zeggen en brengen voort de ondaad en de moord,de groot ondaad en de moord hij spreken te zijn lieden voort,Karel ende Elegast,Moriaen,0.250320
1131,Dadi mi lachter onuerdient Ghi hadt verloren menigen vrient,Men doet mi lachter onverdient Quame enech Gods vrient,doen ik lachter onverdiend gij hebben verliezen menig vriend,men doen ik lachter onverdiend komen enig god vriend,Karel ende Elegast,Moriaen,0.254720
1043,Voer tynghelem in sijn casteel Sijn herte was sonder riueel,Daer hi vant enen casteel Sijn herte was sonder riveel,voor n(prop) in zijn kasteel zijn hart zijn zonder rive,daar hij vinden een kasteel zijn hart zijn zonder rijvel,Karel ende Elegast,Moriaen,0.271022
566,Ick sel v segghen minen name Ist v wille ende bequame,Maer wistic uwer beider name Ende u wille ware ende bequame,ik zullen gij zeggen mijn naam zijn het uw wil en bekwaam,maar ik weten uw beide naam en uw wil zijn en bekwaam,Karel ende Elegast,Moriaen,0.280056
1255,Om datti aen legger was Hi beette neder int gras,Sint dat ic geboren was Ende Gariet hi beette int gras,om dat du aan legger zijn hij beten neder het in gras,zijn dat ik geboren zijn en gari's1et hij beten het in gras,Karel ende Elegast,Moriaen,0.304129
